In [1]:
import json
import pandas as pd

# Loading data

In [2]:
interactions = pd.read_json('data/goodreads_interactions_comics_graphic.json', lines=True)

interactions.columns

Index(['user_id', 'book_id', 'review_id', 'is_read', 'rating',
       'review_text_incomplete', 'date_added', 'date_updated', 'read_at',
       'started_at'],
      dtype='object')

In [26]:
work_id_book_id_dict = pd.read_csv('work_id_book_id_dict.csv')
work_id_book_id_dict

,work_id,book_id
0,797,"[3352661, 19545707, 620786, 32620655, 10478927..."
1,1027,"[89816, 9172685]"
2,1315,"[863050, 9697547, 412264, 321821, 17790324, 34..."
3,1398,[7552]
4,2647,"[825059, 437382, 6886316, 13405305, 1023049, 2..."
...,...,...
62939,58292657,[8561995]
62940,58346678,[13368608]
62941,58356540,"[16230999, 19091791]"
62942,58359101,[22016211]


# Getting work_ids for each book_id

In [43]:
import ast
work_id_book_id_dict['book_id'] = work_id_book_id_dict['book_id'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
book_id_to_work_id = work_id_book_id_dict.explode('book_id').sort_values('book_id')

book_id_to_work_id

,work_id,book_id
13,4676,374
62278,57154110,450
14,4841,586
15,4900,649
3825,406956,690
...,...,...
62915,58189202,36478518
62917,58190888,36480175
30090,19072429,36493971
31182,20132139,36494299


In [47]:
# Verify that there are no duplicate book_id
book_id_to_work_id.groupby('book_id').count().value_counts()

work_id
1          89411
Name: count, dtype: int64

# Adding work_ids to interactions

In [49]:
interactions['work_id'] = interactions['book_id'].map(book_id_to_work_id.set_index('book_id')['work_id'])

In [51]:
interactions.drop(columns=['review_text_incomplete'], inplace=True)

In [52]:
interactions

,user_id,book_id,review_id,is_read,rating,date_added,date_updated,read_at,started_at,work_id
0,8842281e1d1347389f2ab93d60773d4d,836610,6b4db26aafeaf0da77c7de6214331e1e,False,0,Mon Aug 21 12:11:00 -0700 2017,Mon Aug 21 12:11:00 -0700 2017,,,237194
1,8842281e1d1347389f2ab93d60773d4d,7648967,99b27059f711c37de8f90ee8e4dc0d1b,False,0,Fri Feb 24 08:59:44 -0800 2017,Fri Feb 24 08:59:44 -0800 2017,,,10202907
2,8842281e1d1347389f2ab93d60773d4d,15704307,cb944d94854df5afd22210bb0aa0c903,False,0,Wed May 20 21:28:56 -0700 2015,Wed May 20 21:28:57 -0700 2015,,,19113524
3,8842281e1d1347389f2ab93d60773d4d,6902644,2711bac2a8cc600dae1590a6ca0edb34,False,0,Sun Jun 01 17:25:23 -0700 2014,Sun Jun 01 17:25:23 -0700 2014,,,7126174
4,8842281e1d1347389f2ab93d60773d4d,9844623,b72979076d1cded25dded922195e5b1c,False,0,Sun Sep 02 08:45:08 -0700 2012,Sun Sep 02 08:45:08 -0700 2012,,,14735455
...,...,...,...,...,...,...,...,...,...,...
7347625,bd3ac2e547a4f521927056cbd6bb5c2f,1484167,bbaef6acb7e6e1a165e3b52c1ad56c67,True,5,Thu Jun 07 08:39:53 -0700 2012,Thu Jun 07 08:39:53 -0700 2012,,,1988464
7347626,bd3ac2e547a4f521927056cbd6bb5c2f,122451,cdd31f160c2db01232b9998600940d0c,True,5,Wed Jun 06 15:17:39 -0700 2012,Wed Jun 06 15:17:39 -0700 2012,,,2150655
7347627,6384a10d5611945b26b25c971f348fa4,85574,2047dd9f2fc589340a9ec011fbcff490,True,3,Mon Nov 05 10:37:35 -0800 2012,Mon Nov 05 10:37:48 -0800 2012,,,296119
7347628,e9aea57d21cdf9d91a65687d59518924,15197,e8df0a577d4fd440b223c54cdf641dc0,True,5,Mon Jun 04 15:45:50 -0700 2012,Mon Jun 04 15:45:50 -0700 2012,,,1782551


# Export to csv

In [54]:
interactions.to_csv('data/goodreads_interactions_comics_graphic_cleaned.csv', index=False)